In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export yresnet

Cells will be exported to model_constructor.yresnet,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# YResnet.

> Yet another resnet model.

In [ ]:
%nbdev_hide
# from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
%nbdev_export
import torch.nn as nn
import sys, torch
from functools import partial
from collections import OrderedDict
from model_constructor.layers import *
from model_constructor.net import Net

# Utils

In [ ]:
# %nbdev_export
# act_fn = nn.ReLU(inplace=True)

# def init_cnn(m):
#     if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
#     if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
#     for l in m.children(): init_cnn(l)


# YResBlock

In [ ]:
%nbdev_export
# YResBlock - former NewResBlock.
class YResBlock(nn.Module):
    '''YResBlock. Reduce by pool instead of stride 2'''
    def __init__(self, expansion, ni, nh, stride=1, 
                 conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False,sym=False, groups=1):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=1, act_fn=act_fn, bn_1st=bn_1st)), # stride 1 !!!
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ] if expansion == 1 else [
                   (f"conv_0",conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1",conv_layer(nh, nh, 3, stride=1, act_fn=act_fn, bn_1st=bn_1st, groups=int(nh/groups))), # stride 1 !!!
                   (f"conv_2",conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.merge =act_fn

    def forward(self, x): 
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

In [ ]:
%nbdev_hide
bl = YResBlock(1,64,64,sa=True)
bl

YResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (merge): ReLU(inplace=True)
)

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
%nbdev_hide
bl = YResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False)
bl

YResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [ ]:
%nbdev_hide
bl = YResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False, groups=4)
bl

YResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [ ]:
# Stem, Body, Head

# %nbdev_export
# def _make_stem(self):
#         stem = [(f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
#                     stride=2 if i==0 else 1, 
#                     bn_layer=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True,
#                     act_fn=self.act_fn, bn_1st=self.bn_1st))
#                 for i in range(len(self.stem_sizes)-1)]
#         stem.append(('stem_pool', self.stem_pool))
#         if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
#         return nn.Sequential(OrderedDict(stem))

In [ ]:
# %nbdev_export
# def _make_layer(self,expansion,ni,nf,blocks,stride,sa):
#         return nn.Sequential(OrderedDict(
#             [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
#                     stride if i==0 else 1, sa=sa if i==blocks-1 else False,
#                     conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool,
#                                     zero_bn=self.zero_bn, bn_1st=self.bn_1st, groups=self.groups))
#               for i in range(blocks)]))

In [ ]:
# %nbdev_export
# def _make_body(self):
#         blocks = [(f"l_{i}"%nbdev_export
# def _make_layer(self,expansion,ni,nf,blocks,stride,sa):
#         return nn.Sequential(OrderedDict(
#             [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
#                     stride if i==0 else 1, sa=sa if i==blocks-1 else False,
#                     conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool,
#                                     zero_bn=self.zero_bn, bn_1st=self.bn_1st, groups=self.groups))
#               for i in range(blocks)])), self._make_layer(self,self.expansion, 
#                         self.block_szs[i], self.block_szs[i+1], l, 
#                         1 if i==0 else 2, self.sa if i==0 else False))
#                   for i,l in enumerate(self.layers)]
#         return nn.Sequential(OrderedDict(blocks))

In [ ]:
# %nbdev_export
# def _make_head(self):
#         head = [('pool', nn.AdaptiveAvgPool2d(1)),
#                 ('flat', Flatten()),
#                 ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
#         return nn.Sequential(OrderedDict(head))

# YResNet constructor.

In [ ]:
# %nbdev_export
# # v8
# class Net():
#     def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000, name='Net',
#                 act_fn=nn.ReLU(inplace=True), pool = nn.AvgPool2d(2, ceil_mode=True), sa=0):
#         super().__init__()
#         self.name = name
#         self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers # todo setter for expansion
#         self.act_fn, self.pool, self.sa = act_fn, pool, sa
#         self.groups = 1
        
#         self.stem_sizes = [c_in,32,32,64]
#         self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.stem_bn_end = False
#         self.block = ResBlock
#         self.norm = nn.BatchNorm2d
#         self.bn_1st = True
#         self.zero_bn=True
#         self.conv_layer = ConvLayer
#         self._init_cnn = init_cnn
#         self._make_stem = _make_stem
#         self._make_layer = _make_layer
#         self._make_body = _make_body
#         self._make_head = _make_head
        
        
#     @property
#     def block_szs(self):
#         return [self.stem_sizes[-1]//self.expansion,64,128,256,512] +[256]*(len(self.layers)-4) 

#     @property
#     def stem(self):
#         return self._make_stem(self)
#     @property
#     def head(self):
#         return self._make_head(self)
# #     @property
# #     def _make_layer(self):
# #         return self.__make_layer(self)
#     @property
#     def body(self):
#         return self._make_body(self)
    
#     def __call__(self):
#         model = nn.Sequential(OrderedDict([
#             ('stem', self.stem),
#             ('body', self.body),
#             ('head', self.head)
#         ]))
#         self._init_cnn(model)
#         model.extra_repr = lambda : f"model {self.name}"
#         return model
#     def __repr__(self):
#         return f" constr {self.name}\n expansion: {self.expansion}, sa: {self.sa}, groups: {self.groups}\n stem sizes: {self.stem_sizes}\n body sizes {self.block_szs}"

In [ ]:
model  = Net(block=YResBlock, name='yresnet')

In [ ]:
model

yresnet constructor
 expansion: 1, sa: 0, groups: 1
 stem sizes: [3, 32, 32, 64]
 body sizes [64, 64, 128, 256, 512]

In [ ]:
model.block_szs

[64, 64, 128, 256, 512]

In [ ]:
%nbdev_collapse_output
model.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = model.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
model.bn_1st = False

In [ ]:
model.act_fn =nn.LeakyReLU(inplace=True)

In [ ]:
model.sa = True

In [ ]:
%nbdev_collapse_output
model.body.l_1

Sequential(
  (bl_0): YResBlock(
    (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (idconv): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (merge): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): YResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(128

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
%nbdev_hide
model.body.l_0

Sequential(
  (bl_0): YResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (merge): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): YResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
      

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
%nbdev_hide
model.groups = 4
model.expansion = 4
model.body.l_0

Sequential(
  (bl_0): YResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (idconv): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [ ]:
%nbdev_hide
model.groups = 1
model.expansion = 1

In [ ]:
# model = Net(expansion=4)
model.expansion = 4

In [ ]:
%nbdev_hide
model.stem_bn_end = True
model.stem_stride_on = 1

In [ ]:
%nbdev_hide
model.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
%nbdev_hide
model.body.l_1.bl_0

YResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = model.body.l_1.bl_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [ ]:
%nbdev_hide
model.stem_bn_end = False

In [ ]:
%nbdev_collapse_input
m = model()

In [ ]:
%nbdev_hide
m

Sequential(
  model yresnet
  (stem): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (

In [ ]:
%nbdev_collapse_output
m.body

Sequential(
  (l_0): Sequential(
    (bl_0): YResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (idconv): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = m(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


## YResNet constructor

In [ ]:
%nbdev_export
yresnet34  = partial(Net, block=YResBlock, expansion=1, layers=[3, 4,  6, 3], name='yresnet34')
yresnet50  = partial(Net, block=YResBlock, expansion=4, layers=[3, 4,  6, 3], name='yresnet50')

In [ ]:
m = yresnet50(c_out=10)

In [ ]:
m, m.c_out

(yresnet50 constructor
  expansion: 4, sa: 0, groups: 1
  stem sizes: [3, 32, 32, 64]
  body sizes [16, 64, 128, 256, 512],
 10)

# end
model_constructor
by ayasyrev

In [ ]:
%nbdev_hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted 03_xresnet.ipynb.
Converted 04_Net.ipynb.
Converted 05_Twist.ipynb.
Converted 06_YResnet.ipynb.
Converted index.ipynb.
